## Attention ANN

| Iteration | Dropout | L2 Reg. | BatchNorm | EarlyStopping | LR Scheduler      | Scaling                              | Physics Features | Notes                               |
| --------- | ------- | ------- | --------- | ------------- | ----------------- | ------------------------------------ | ---------------- | ----------------------------------- |
| **1**     | 0.2     | 1e-4      | Yes        | Yes            | ReduceLROnPlateau     | StdScaler (X) + Ringing→MinMax       | Yes              | Best overall: 11 positive R², interpretable attentionANN                    |


### ITERATION 1

In [3]:
# ==================== ATTENTION ANN (Best Iteration) ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = "aiteration_2"
os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(
    r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_25percent_balanced.csv"
)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID', 'MOSFET', 'Part_Number']

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number'] == UNSEEN_PART].copy()

# ============== PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row.get("VDS_max",0) - row.get("Vbus",0)
    undershoot_est = - row.get("VGS_th_min",0)
    dVdt_est = row.get("VDS_max",0)/row.get("Tp1",1e-9)
    dIdt_est = row.get("ID_max_25C",0)/row.get("Tp1",1e-9)
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df, test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = \
        df_.apply(compute_physics_features, axis=1)

physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features

# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS])

# Separate physics cols
phys_idx = [INPUT_COLUMNS.index(c) for c in physics_features]
X_train_phys = X_train_all[:, phys_idx]
X_test_phys  = X_test_all[:, phys_idx]
X_train_main = np.delete(X_train_all, phys_idx, axis=1)
X_test_main  = np.delete(X_test_all, phys_idx, axis=1)

# ============== SCALE OUTPUTS (per target) ==============
output_scalers, y_train_scaled, y_test_scaled = {}, [], []
for col in TARGET_COLUMNS:
    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    y_train_scaled.append(scaler.fit_transform(train_df[[col]]).flatten())
    y_test_scaled.append(scaler.transform(test_df[[col]]).flatten())
    output_scalers[col] = scaler
y_train_scaled = np.array(y_train_scaled).T
y_test_scaled  = np.array(y_test_scaled).T

# ============== SPLIT TRAIN/VAL ==============
Xtr_main, Xval_main, Xtr_phys, Xval_phys, ytr, yval = train_test_split(
    X_train_main, X_train_phys, y_train_scaled,
    test_size=0.15, random_state=SEED
)

# ============== BUILD ATTENTION ANN ==============
def build_attention_ann(input_dim_main, input_dim_phys, output_dim, l2_reg=1e-4, dropout_rate=0.2):
    inp_main = layers.Input(shape=(input_dim_main,), name="main_inputs")
    inp_phys = layers.Input(shape=(input_dim_phys,), name="physics_inputs")

    # Main backbone
    x_main = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg))(inp_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main)
    x_main = layers.Dropout(dropout_rate)(x_main)

    x_main = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg))(x_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main)
    x_main = layers.Dropout(dropout_rate)(x_main)

    # Physics branch
    x_phys = layers.Dense(32, activation="relu")(inp_phys)

    # Attention mechanism
    concat = layers.Concatenate()([x_main, x_phys])
    attn = layers.Dense(concat.shape[-1], activation="softmax")(concat)
    attended = layers.multiply([concat, attn])

    # Multi-head outputs
    outputs = []
    for col in TARGET_COLUMNS:
        h = layers.Dense(64, activation="relu")(attended)
        h = layers.Dropout(dropout_rate)(h)
        h = layers.Dense(32, activation="relu")(h)
        out = layers.Dense(1, activation="linear", name=col)(h)
        outputs.append(out)

    model = models.Model(inputs=[inp_main, inp_phys], outputs=outputs)
    return model

model = build_attention_ann(Xtr_main.shape[1], Xtr_phys.shape[1], len(TARGET_COLUMNS))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=["mse"]*len(TARGET_COLUMNS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr  = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=5)

history = model.fit(
    [Xtr_main, Xtr_phys],
    [ytr[:,i] for i in range(len(TARGET_COLUMNS))],
    validation_data=([Xval_main,Xval_phys],[yval[:,i] for i in range(len(TARGET_COLUMNS))]),
    epochs=120, batch_size=256, callbacks=[early_stop, reduce_lr], verbose=1
)

model.save(f"{BASE_DIR}/models/attention_ann.h5")

# ============== LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("MSE Loss")
plt.title("Train vs Validation Loss")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# ============== EVALUATION ==============
def evaluate_and_save(X_main, X_phys, y_scaled, name, positive_only=False):
    y_pred_scaled_list = model.predict([X_main, X_phys])
    results = []
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:,i].reshape(-1,1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled_list[i].reshape(-1,1)).flatten()
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        results.append((col, r2, rmse))
    df_results = pd.DataFrame(results, columns=["Target","R2","RMSE"]).sort_values("R2",ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"]>0]
    print(f"\n{name}:\n",df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)
    return df_results

evaluate_and_save(Xtr_main,Xtr_phys,ytr,"train")
evaluate_and_save(Xval_main,Xval_phys,yval,"val")
evaluate_and_save(X_train_main,X_train_phys,y_train_scaled,"test")
evaluate_and_save(X_test_main,X_test_phys,y_test_scaled,"unseen",positive_only=True)


Epoch 1/120
287/287 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - current_fall_time_pulse1_loss: 0.6140 - current_fall_time_pulse1_mae: 0.6029 - current_fall_time_pulse2_loss: 0.6242 - current_fall_time_pulse2_mae: 0.6053 - current_rise_time_pulse1_loss: 0.6335 - current_rise_time_pulse1_mae: 0.5989 - current_rise_time_pulse2_loss: 0.6205 - current_rise_time_pulse2_mae: 0.6360 - loss: 7.5962 - overshoot_pulse_1_loss: 0.8112 - overshoot_pulse_1_mae: 0.6854 - overshoot_pulse_2_loss: 0.6921 - overshoot_pulse_2_mae: 0.6360 - ringing_frequency_MHz_loss: 0.0947 - ringing_frequency_MHz_mae: 0.2080 - undershoot_pulse_1_loss: 0.6688 - undershoot_pulse_1_mae: 0.6190 - undershoot_pulse_2_loss: 0.6915 - undershoot_pulse_2_mae: 0.6255 - voltage_fall_time_pulse1_loss: 0.4948 - voltage_fall_time_pulse1_mae: 0.5177 - voltage_fall_time_pulse2_loss: 0.4708 - voltage_fall_time_pulse2_mae: 0.5026 - voltage_rise_time_pulse1_loss: 0.5409 - voltage_rise_time_pulse1_mae: 0.5478 - voltage_rise_time_pulse2_loss: 0.6258 

2294/2294 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step

train:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.998891  2.286280e-10
2   voltage_fall_time_pulse1  0.998857  2.324009e-10
7   current_fall_time_pulse2  0.997369  6.034028e-10
6   current_fall_time_pulse1  0.997121  6.331242e-10
12     ringing_frequency_MHz  0.995332  2.256086e+00
0   voltage_rise_time_pulse1  0.989917  4.068335e-10
10        undershoot_pulse_1  0.983050  1.783756e+00
11        undershoot_pulse_2  0.982822  1.796712e+00
8          overshoot_pulse_1  0.966527  2.237598e+00
4   current_rise_time_pulse1  0.936372  1.239511e-08
9          overshoot_pulse_2  0.935248  6.435629e+00
1   voltage_rise_time_pulse2  0.921194  1.130460e-09
5   current_rise_time_pulse2  0.912721  7.420115e-09
405/405 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step

val:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.998885  2.281617e-10
2   voltage_fall_time_pulse1  0.998862  2.306908e-10

,Target,R2,RMSE
7,current_fall_time_pulse2,0.903889,3.456598e-09
6,current_fall_time_pulse1,0.897723,3.590420e-09
8,overshoot_pulse_1,0.718871,7.170268e+00
9,overshoot_pulse_2,0.642172,1.239294e+01
11,undershoot_pulse_2,0.570570,6.384569e+00
10,undershoot_pulse_1,0.556023,6.536107e+00
5,current_rise_time_pulse2,0.461157,1.230650e-08
1,voltage_rise_time_pulse2,0.371650,3.698144e-09
0,voltage_rise_time_pulse1,0.191292,1.548524e-09


In [5]:
# ==================== ATTENTION ANN (Best Iteration) ====================
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, regularizers
import matplotlib.pyplot as plt

# ============== SETTINGS ==============
SEED = 42
UNSEEN_PART = 'C2M0040120D'
BASE_DIR = "iteration_1"
os.makedirs(f"{BASE_DIR}/r2_rmse_tables", exist_ok=True)
os.makedirs(f"{BASE_DIR}/train_val_loss_curves", exist_ok=True)
os.makedirs(f"{BASE_DIR}/models", exist_ok=True)

# ============== LOAD DATA ==============
df = pd.read_csv(
    r"C:\Users\pc\Desktop\Neural_Network_Models\merged_train_5_MOSFETs_10percent_balanced.csv"
)

TARGET_COLUMNS = [
    'voltage_rise_time_pulse1', 'voltage_rise_time_pulse2',
    'voltage_fall_time_pulse1', 'voltage_fall_time_pulse2',
    'current_rise_time_pulse1', 'current_rise_time_pulse2',
    'current_fall_time_pulse1', 'current_fall_time_pulse2',
    'overshoot_pulse_1', 'overshoot_pulse_2',
    'undershoot_pulse_1', 'undershoot_pulse_2',
    'ringing_frequency_MHz'
]
DROP_COLUMNS = ['DeviceID', 'MOSFET', 'Part_Number']

# ============== SPLIT SEEN/UNSEEN DEVICES ==============
seen_parts = [p for p in df['Part_Number'].unique().tolist() if p != UNSEEN_PART]
train_df = df[df['Part_Number'].isin(seen_parts)].copy()
test_df  = df[df['Part_Number'] == UNSEEN_PART].copy()

# ============== PHYSICS FEATURES ==============
def compute_physics_features(row):
    L_eq = row[['Ls4','Ls5','Ls6','Ls7','Ls8','Ls9','Ls10','Ls11']].sum()
    C_eq = row.get("Coss", 1e-12)
    f_res = 1/(2*np.pi*np.sqrt(L_eq*C_eq))/1e6 if L_eq>0 and C_eq>0 else 0
    overshoot_est = row.get("VDS_max",0) - row.get("Vbus",0)
    undershoot_est = - row.get("VGS_th_min",0)
    dVdt_est = row.get("VDS_max",0)/row.get("Tp1",1e-9)
    dIdt_est = row.get("ID_max_25C",0)/row.get("Tp1",1e-9)
    return pd.Series([f_res, overshoot_est, undershoot_est, dVdt_est, dIdt_est])

for df_ in [train_df, test_df]:
    df_[['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']] = \
        df_.apply(compute_physics_features, axis=1)

physics_features = ['f_resonance','overshoot_est','undershoot_est','dVdt_est','dIdt_est']
INPUT_COLUMNS = [c for c in df.columns if c not in TARGET_COLUMNS + DROP_COLUMNS] + physics_features

# ============== SCALE INPUTS ==============
input_scaler = StandardScaler()
input_scaler.fit(pd.concat([train_df[INPUT_COLUMNS], test_df[INPUT_COLUMNS]]))
X_train_all = input_scaler.transform(train_df[INPUT_COLUMNS])
X_test_all  = input_scaler.transform(test_df[INPUT_COLUMNS])

# Separate physics cols
phys_idx = [INPUT_COLUMNS.index(c) for c in physics_features]
X_train_phys = X_train_all[:, phys_idx]
X_test_phys  = X_test_all[:, phys_idx]
X_train_main = np.delete(X_train_all, phys_idx, axis=1)
X_test_main  = np.delete(X_test_all, phys_idx, axis=1)

# ============== SCALE OUTPUTS (per target) ==============
output_scalers, y_train_scaled, y_test_scaled = {}, [], []
for col in TARGET_COLUMNS:
    scaler = MinMaxScaler() if col == 'ringing_frequency_MHz' else StandardScaler()
    y_train_scaled.append(scaler.fit_transform(train_df[[col]]).flatten())
    y_test_scaled.append(scaler.transform(test_df[[col]]).flatten())
    output_scalers[col] = scaler
y_train_scaled = np.array(y_train_scaled).T
y_test_scaled  = np.array(y_test_scaled).T

# ============== SPLIT TRAIN/VAL ==============
Xtr_main, Xval_main, Xtr_phys, Xval_phys, ytr, yval = train_test_split(
    X_train_main, X_train_phys, y_train_scaled,
    test_size=0.15, random_state=SEED
)

# ============== BUILD ATTENTION ANN ==============
def build_attention_ann(input_dim_main, input_dim_phys, output_dim, l2_reg=1e-4, dropout_rate=0.2):
    inp_main = layers.Input(shape=(input_dim_main,), name="main_inputs")
    inp_phys = layers.Input(shape=(input_dim_phys,), name="physics_inputs")

    # Main backbone
    x_main = layers.Dense(128, kernel_regularizer=regularizers.l2(l2_reg))(inp_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main)
    x_main = layers.Dropout(dropout_rate)(x_main)

    x_main = layers.Dense(64, kernel_regularizer=regularizers.l2(l2_reg))(x_main)
    x_main = layers.BatchNormalization()(x_main); x_main = layers.ReLU()(x_main)
    x_main = layers.Dropout(dropout_rate)(x_main)

    # Physics branch
    x_phys = layers.Dense(32, activation="relu")(inp_phys)

    # Attention mechanism
    concat = layers.Concatenate()([x_main, x_phys])
    attn = layers.Dense(concat.shape[-1], activation="softmax")(concat)
    attended = layers.multiply([concat, attn])

    # Multi-head outputs
    outputs = []
    for col in TARGET_COLUMNS:
        h = layers.Dense(64, activation="relu")(attended)
        h = layers.Dropout(dropout_rate)(h)
        h = layers.Dense(32, activation="relu")(h)
        out = layers.Dense(1, activation="linear", name=col)(h)
        outputs.append(out)

    model = models.Model(inputs=[inp_main, inp_phys], outputs=outputs)
    return model

model = build_attention_ann(Xtr_main.shape[1], Xtr_phys.shape[1], len(TARGET_COLUMNS))

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    loss=["mse"]*len(TARGET_COLUMNS),
    metrics=["mae"]*len(TARGET_COLUMNS)
)

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)
reduce_lr  = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.3, patience=5)

history = model.fit(
    [Xtr_main, Xtr_phys],
    [ytr[:,i] for i in range(len(TARGET_COLUMNS))],
    validation_data=([Xval_main,Xval_phys],[yval[:,i] for i in range(len(TARGET_COLUMNS))]),
    epochs=120, batch_size=256, callbacks=[early_stop, reduce_lr], verbose=1
)

model.save(f"{BASE_DIR}/models/attention_ann.h5")

# ============== LOSS CURVE ==============
plt.figure()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel("Epoch"); plt.ylabel("MSE Loss")
plt.title("Train vs Validation Loss")
plt.legend(); plt.savefig(f"{BASE_DIR}/train_val_loss_curves/loss.png"); plt.close()

# ============== EVALUATION ==============
def evaluate_and_save(X_main, X_phys, y_scaled, name, positive_only=False):
    y_pred_scaled_list = model.predict([X_main, X_phys])
    results = []
    for i, col in enumerate(TARGET_COLUMNS):
        y_true = output_scalers[col].inverse_transform(y_scaled[:,i].reshape(-1,1)).flatten()
        y_pred = output_scalers[col].inverse_transform(y_pred_scaled_list[i].reshape(-1,1)).flatten()
        r2 = r2_score(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        results.append((col, r2, rmse))
    df_results = pd.DataFrame(results, columns=["Target","R2","RMSE"]).sort_values("R2",ascending=False)
    if positive_only:
        df_results = df_results[df_results["R2"]>0]
    print(f"\n{name}:\n",df_results)
    df_results.to_csv(f"{BASE_DIR}/r2_rmse_tables/{name}.csv", index=False)
    return df_results

evaluate_and_save(Xtr_main,Xtr_phys,ytr,"train")
evaluate_and_save(Xval_main,Xval_phys,yval,"val")
evaluate_and_save(X_train_main,X_train_phys,y_train_scaled,"test")
evaluate_and_save(X_test_main,X_test_phys,y_test_scaled,"unseen",positive_only=True)


Epoch 1/120
29/29 ━━━━━━━━━━━━━━━━━━━━ 15s 67ms/step - current_fall_time_pulse1_loss: 0.9641 - current_fall_time_pulse1_mae: 0.7834 - current_fall_time_pulse2_loss: 0.9693 - current_fall_time_pulse2_mae: 0.7897 - current_rise_time_pulse1_loss: 0.9819 - current_rise_time_pulse1_mae: 0.7920 - current_rise_time_pulse2_loss: 0.9802 - current_rise_time_pulse2_mae: 0.8353 - loss: 11.8487 - overshoot_pulse_1_loss: 0.9565 - overshoot_pulse_1_mae: 0.7655 - overshoot_pulse_2_loss: 0.9685 - overshoot_pulse_2_mae: 0.7827 - ringing_frequency_MHz_loss: 0.2386 - ringing_frequency_MHz_mae: 0.3981 - undershoot_pulse_1_loss: 0.9566 - undershoot_pulse_1_mae: 0.7757 - undershoot_pulse_2_loss: 0.9663 - undershoot_pulse_2_mae: 0.7813 - voltage_fall_time_pulse1_loss: 0.9705 - voltage_fall_time_pulse1_mae: 0.8106 - voltage_fall_time_pulse2_loss: 0.9654 - voltage_fall_time_pulse2_mae: 0.8119 - voltage_rise_time_pulse1_loss: 0.9694 - voltage_rise_time_pulse1_mae: 0.7898 - voltage_rise_time_pulse2_loss: 0.9461 -

230/230 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step

train:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.998036  3.032721e-10
2   voltage_fall_time_pulse1  0.997571  3.377060e-10
7   current_fall_time_pulse2  0.994889  8.365179e-10
6   current_fall_time_pulse1  0.994884  8.398879e-10
12     ringing_frequency_MHz  0.994777  2.374513e+00
11        undershoot_pulse_2  0.977702  2.072213e+00
10        undershoot_pulse_1  0.976679  2.115020e+00
0   voltage_rise_time_pulse1  0.976515  6.174913e-10
8          overshoot_pulse_1  0.954413  2.458395e+00
4   current_rise_time_pulse1  0.936501  1.230487e-08
9          overshoot_pulse_2  0.927821  6.648456e+00
1   voltage_rise_time_pulse2  0.911004  1.146221e-09
5   current_rise_time_pulse2  0.896020  7.952848e-09
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step

val:
                       Target        R2          RMSE
3   voltage_fall_time_pulse2  0.997560  3.483185e-10
2   voltage_fall_time_pulse1  0.997188  3.745791e-10
6  

,Target,R2,RMSE
6,current_fall_time_pulse1,0.931825,2.932103e-09
7,current_fall_time_pulse2,0.923373,3.087279e-09
8,overshoot_pulse_1,0.544395,8.574716e+00
9,overshoot_pulse_2,0.437781,1.489661e+01
11,undershoot_pulse_2,0.423919,7.228575e+00
10,undershoot_pulse_1,0.407673,7.382952e+00
0,voltage_rise_time_pulse1,0.320046,1.437613e-09
3,voltage_fall_time_pulse2,0.288584,1.964124e-09
5,current_rise_time_pulse2,0.287961,1.400704e-08
1,voltage_rise_time_pulse2,0.281665,3.733709e-09
